In [1]:
!pip install gym

In [2]:
!git clone https://github.com/ClementRomac/gym-tictactoe
%cd gym-tictactoe
!python setup.py install

Cloning into 'gym-tictactoe'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 42 (delta 3), reused 1 (delta 0), pack-reused 30
Unpacking objects: 100% (42/42), done.
/content/gym-tictactoe
running install
running bdist_egg
running egg_info
creating gym_tictactoe.egg-info
writing gym_tictactoe.egg-info/PKG-INFO
writing dependency_links to gym_tictactoe.egg-info/dependency_links.txt
writing requirements to gym_tictactoe.egg-info/requires.txt
writing top-level names to gym_tictactoe.egg-info/top_level.txt
writing manifest file 'gym_tictactoe.egg-info/SOURCES.txt'
writing manifest file 'gym_tictactoe.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/gym_tictactoe
copying gym_tictactoe/__init__.py -> build/lib/gym_tictactoe
copying gym_tictactoe/tic_tac_toe.py -> build/lib/g

In [3]:
import gym
import gym_tictactoe
import random
import numpy as np
env = gym.make('TicTacToe-v1', symbols=[-1, 1], board_size=3, win_size=3) 

In [4]:
def one_hot_encoding(action):
    len_action = 9
    one_hot_vector = [0]* len_action
    index = action
    one_hot_vector[index]=1
    return one_hot_vector

In [5]:
def randomAction(s, n=None):
    return random.randrange(0, 9)

In [6]:
def us0_data_preparation(N, K, f0, f1, render=False, extra_f0 = randomAction, extra_f1 = randomAction):
    game_data = []
    for i in range(N):
        game_steps = []
        state = env.reset()
        user = 0
        reward = 0
        done = False
        score = 0
        while not done:
            if render: env.render(mode=None)
            if user == 0:
                us0_action = f0(state)
                nTry = 2
                while state[us0_action] != 0:
                    us0_action = extra_f0(state, nTry)
                    nTry += 1
                game_steps.append((state, us0_action))
                state, reward, done, info = env.step(us0_action, -1)
            elif user == 1:
                us1_action = f1(state)
                nTry = 2
                while state[us1_action] != 0:
                    us1_action = extra_f1(state, nTry)
                    nTry += 1
                _, _, done, info = env.step(us1_action, 1)
            
            if not done:
                user = 0 if user == 1 else 1
            else:
                if reward == 10:
#                     print("Draw !")
                    score = reward
                elif reward == -20:
                    if user == 0:
                        score = reward
#                         print("Random wins ! AI Reward : " + str(reward))
                    elif user == 1:
                        score = -reward
#                         print("AI wins ! AI Reward : " + str(-reward))
                elif reward == 20:
                    if user == 0:
                        score = reward
#                         print("AI wins ! AI Reward : " + str(reward))
                    elif user == 1:
                        score = -reward
#                         print("Random wins ! AI Reward : " + str(reward))
        game_data.append((score, game_steps))
    game_data.sort(key=lambda s:-s[0])
    
    training_set = []
    for i in range(K):
        for step in game_data[i][1]:
            training_set.append((step[0], one_hot_encoding(step[1])))
    print("Complete")
    if render:
        for i in game_data:
            print("Score: {0}".format(i[0]))
    return training_set
    

In [7]:
def us1_data_preparation(N, K, f0, f1, render=False, extra_f0 = randomAction, extra_f1 = randomAction):
    game_data = []
    for i in range(N):
        game_steps = []
        state = env.reset()
        user = 0
        reward = 0
        done = False
        score = 0
        while not done:
            if render: env.render(mode=None)
            if user == 0:
                us0_action = f0(state)
                nTry = 2
                while state[us0_action] != 0:
                    us0_action = extra_f0(state, nTry)
                    nTry += 1
                _, _, done, info = env.step(us0_action, -1)
            elif user == 1:
                us1_action = f1(state)
                nTry = 2
                while state[us1_action] != 0:
                    us1_action = extra_f1(state, nTry)
                    nTry += 1
                game_steps.append((state, us1_action))
                state, reward, done, info = env.step(us1_action, 1)

            
            if not done:
                user = 0 if user == 1 else 1
            else:
                if reward == 10:
#                     print("Draw !")
                    score = reward
                elif reward == -20:
                    if user == 0:
                        score = -reward
#                         print("Random wins ! AI Reward : " + str(reward))
                    elif user == 1:
                        score = reward
#                         print("AI wins ! AI Reward : " + str(-reward))
                elif reward == 20:
                    if user == 0:
                        score = -reward
#                         print("AI wins ! AI Reward : " + str(reward))
                    elif user == 1:
                        score = reward
#                         print("Random wins ! AI Reward : " + str(reward))

        game_data.append((score, game_steps))
    game_data.sort(key=lambda s:-s[0])
    
    training_set = []
    for i in range(K):
        for step in game_data[i][1]:
            training_set.append((step[0], one_hot_encoding(step[1])))
    print("Complete")
    if render:
        for i in game_data:
            print("Score: {0}".format(i[0]))
    return training_set
    

In [8]:
us0_training_data= us0_data_preparation(1000, 50, randomAction, randomAction)

Complete


In [9]:
us1_training_data= us1_data_preparation(1000, 50, randomAction, randomAction)

Complete


In [10]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [11]:
def build_model():
    model = Sequential()
    model.add(Dense(128, input_dim=9, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    model.compile(loss='mse', optimizer=Adam())
    return model

In [12]:
def train_model(model, training_set):
    X = np.array([i[0] for i in training_set]).reshape(-1, 9)
    y = np.array([i[1] for i in training_set]).reshape(-1, 9)
    model.fit(X, y, epochs=10)

In [13]:
def predict_or_random():
    res = random.randint(1, 10)
    if res % 3 != 0:
        return True
    else:
        return False

In [14]:
def us0_predictor(s):
    if predict_or_random():
        return np.argmax(us0_model.predict(np.array(s).reshape(-1, 9))[0])
    else:
        return random.randint(0, 8)

def us1_predictor(s):
    if predict_or_random():
        return np.argmax(us1_model.predict(np.array(s).reshape(-1, 9))[0])
    else:
        return random.randint(0, 8)

In [15]:
def extra_us0_predictor(s, n):
    lst = us0_model.predict(np.array(s).reshape(-1, 9))[0]
    for i in range(n-1):
        iMax = np.argmax(lst)
        lst[iMax] = -1

    if predict_or_random():
        return np.argmax(lst)
    else:
        ret = random.randint(0, 8)
        while s[ret] != 0:
            ret = random.randint(0, 8)
        return ret

def extra_us1_predictor(s, n):
    lst = us1_model.predict(np.array(s).reshape(-1, 9))[0]
    for i in range(n-1):
        iMax = np.argmax(lst)
        lst[iMax] = -1
    
    if predict_or_random():
        return np.argmax(lst)
    else:
        ret = random.randint(0, 8)
        while s[ret] != 0:
            ret = random.randint(0, 8)
        return ret

In [16]:
with tf.device("gpu:0"):
    N = 1000
    K = 50
    us0_model = build_model()
    train_model(us0_model, us0_training_data)
    us1_model = build_model()
    train_model(us1_model, us1_training_data)
    self_play_count = 20

    

    for i in range(self_play_count):
        print("Self playing count : " + str(i))
        K = (N//9 + K)//2
        us0_training_data = us0_data_preparation(N, K, us0_predictor, us1_predictor, extra_f0=extra_us0_predictor, extra_f1=extra_us1_predictor)
        us1_training_data = us1_data_preparation(N, K, us0_predictor, us1_predictor, extra_f0=extra_us0_predictor, extra_f1=extra_us1_predictor)
        train_model(us0_model, us0_training_data)
        train_model(us1_model, us1_training_data)
        print("Complete!")

Epoch 1/10
7/7 [==============================] - 2s 3ms/step - loss: 0.0982
Epoch 2/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0968
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0953
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0943
Epoch 5/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0936
Epoch 6/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0924
Epoch 7/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0925
Epoch 8/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0925
Epoch 9/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0912
Epoch 10/10
7/7 [==============================] - 0s 3ms/step - loss: 0.0910
Epoch 1/10
6/6 [==============================] - 0s 3ms/step - loss: 0.0991
Epoch 2/10
6/6 [==============================] - 0s 2ms/step - loss: 0.0971
Epoch 3/10
6/6 [==============================] - 0s 2ms/step - loss: 0.095

In [17]:
!pip install -q pyyaml h5py

In [18]:
checkpoint_path0 = "/content/drive/MyDrive/Colab Notebooks/Reinforcement Learning/us0_weight/us0.ckpt"
us0_model.save_weights(checkpoint_path0)

In [19]:
checkpoint_path1 = "/content/drive/MyDrive/Colab Notebooks/Reinforcement Learning/us1_weight/us1.ckpt"
us1_model.save_weights(checkpoint_path1)